<a href="https://colab.research.google.com/github/Asal2/GenAI/blob/main/RAG_Pipeline_for_Job_Data_Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU langchain-openai langchain langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.5/449.5 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 12.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
llm_provider = "openai"

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
def get_llm(model=None, temperature=0):
  if llm_provider == "openai":
    return ChatOpenAI(
        api_key=openai_api_key,
        model=model or "gpt-4.1",
        temperature=temperature
      )


In [ ]:
response = get_llm().invoke("what is AI")
response.content

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
import pandas as pd
import re


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/dataset/LF_Jobs.csv")


# Access the last column (column 9)
last_column = df.iloc[:, -1]  # or df[df.columns[-1]]

print(last_column)

0      <p><b>Job Description:</b><br><br>At Bank of A...
1      <p>Wipro Limited (NYSE: WIT, BSE: 507685, NSE:...
2      <p>Eaton's Digital Employee Experience (DEX) t...
3      <p><strong>The Role…</strong></p>\n\n<p>We are...
4      <p><b>Job Summary</b><br><br><b> Job Descripti...
                             ...                        
995    <p><strong>Job Profile:</strong><br><br>Join o...
996    <p>Job Description<br><br><strong>Associate &a...
997    <p><b>Position Summary...</b><br><br>Why do pe...
998    <p>At CVS Health, we're building a world of he...
999    <p><b>Responsibilities</b><br><br> TikTok will...
Name: Job Description, Length: 1000, dtype: object


In [ ]:
def clean_text(text):
  if pd.isna(text):
    return ""
  text = re.sub(r"<.*?>", " ", text)
  text = text.lower()
  text = re.sub(r"\s+", " ", text).strip()
  return text

In [ ]:
df["cleaned description"] = df.iloc[:,-1].apply(clean_text)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=25,
    separators=["\n\n", "\n", " ", ""]
)

In [ ]:
all_chunks = []
for _, row in df.iterrows():
  job_id = row["ID"]
  title = row["Job Title"]
  company = row["Company Name"]
  location = row["Job Location"]
  level = row["Job Level"]

  # Split the cleaned description into chunks
  chunks = r_splitter.split_text(row["cleaned description"])

  # Attach metadata to each chunk
  for i, chunk in enumerate(chunks):
    all_chunks.append({
      "id": f"{job_id}_{i}",        # unique chunk ID
      "job_id": job_id,             # original job ID
      "title": title,
      "company": company,
      "location": location,
      "level": level,
      "text": chunk
    })

In [ ]:
for c in all_chunks[:3]:
    print(c)

{'id': 'LF0001_0', 'job_id': 'LF0001', 'title': 'DIR, Equities Quant', 'company': 'Merrill', 'location': 'New York, NY', 'level': 'Mid Level', 'text': "job description: at bank of america, we are guided by a common purpose to help make financial lives better through the power of every connection. we do this by driving responsible growth and delivering for our clients, teammates, communities and shareholders every day. being a great place to work is core to how we drive responsible growth. this includes our commitment to being an inclusive workplace, attracting and developing exceptional talent, supporting our teammates' physical, emotional, and"}
{'id': 'LF0001_1', 'job_id': 'LF0001', 'title': 'DIR, Equities Quant', 'company': 'Merrill', 'location': 'New York, NY', 'level': 'Mid Level', 'text': 'physical, emotional, and financial wellness, recognizing and rewarding performance, and how we make an impact in the communities we serve. bank of america is committed to an in-office culture w

In [ ]:
print(f"total chunks: {len(all_chunks)} chunks")

total chunks: 11543 chunks


In [ ]:
from langchain_openai import OpenAIEmbeddings

In [ ]:
embedding_model = OpenAIEmbeddings(
    api_key=openai_api_key,
    model="text-embedding-3-large"  # or "text-embedding-3-large"
)

In [ ]:
texts = [c["text"] for c in all_chunks]
embeddings = embedding_model.embed_documents(texts)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
!pip install -q faiss-cpu

In [ ]:
import faiss
import numpy as np

embedding_matrix = np.array(embeddings, dtype="float32")

def normalize(vecs):
  norms = np.linalg.norm(vecs, axis=1, keepdims=True)
  return vecs / norms

normalized_embeddings = normalize(embeddings) #or embedding_matrix



# 3. Create FAISS index
dimension = embedding_matrix.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(normalized_embeddings)
print(f"FAISS index has {index.ntotal} vectors")



In [ ]:
# query = "software engineer jobs"
# query_embedding = model.encode([query], convert_to_tensor=False)

# # Search top 5 matches
# k = 5
# distances, indices = index.search(np.array(query_embedding, dtype="float32"), k)

# # Show matching chunks
# for idx in indices[0]:
#     print(all_chunks[idx]["text"])


In [ ]:
def get_openai_embedding(text):
  """Single query embedding"""
  return np.array(embedding_model.embed_query(text), dtype="float32")

def retrieve_faiss_openai(query, index, chunks, k=5):
  # Embed + normalize query
  query_vec = get_openai_embedding(query).reshape(1, -1)
  query_vec = normalize(query_vec)

  # Search
  D, I = index.search(query_vec, k)

  retrieved_docs = []
  for j, i in enumerate(I[0]):
    if i == -1:
        # no result
      continue

    retrieved_docs.append(
        f"[{j+1}] {all_chunks[i]['title']} at {all_chunks[i]['company']} "
        f"({all_chunks[i]['category']}, {all_chunks[i]['date']})"
    )

        # Build context string
  context = "\n".join(retrieved_docs)

  # Prompt LLM
  prompt = ChatPromptTemplate.from_template(
      "You are a helpful assistant that answers job queries.\n"
      "User query: {query}\n\n"
      "Retrieved job postings:\n{context}\n\n"
      "Based on the retrieved jobs, provide a clear and enriched answer for the user."
  )

  llm_response = llm.invoke(prompt.format(query=req.query, context=context))

  return {
      "query": req.query,
      "retrieved_jobs": retrieved_docs,
      "enriched_answer": llm_response.content
  }

  # Return matched chunks with similarity scores
#   results = []
#   for j, i in enumerate(I[0]):
#     results.append({
#       "score": float(D[0][j]),
#       "job_id": chunks[i]["job_id"],
#       "text": chunks[i]["text"]
#     })
#   return results

# # Example usage
# query = "remote software engineer jobs"
# top_results = retrieve_faiss_openai(query, index, all_chunks, k=5)

# for r in top_results:
#     print(f"Score: {r['score']:.3f} | Job ID: {r['job_id']}")
#     print(r['text'], "...\n")


In [ ]:
llm = get_llm(model="gpt-4.1") # Using a faster, modern model like gpt-4o is often a good choice

# 2. Create a prompt template
# This instructs the LLM on how to use the retrieved context
prompt_template = """
Answer the user's question based only on the following context.
If the context doesn't contain the answer, state that you don't have enough information.
Do not make up information.

Context:
{context}

Question:
{question}

Answer:
"""

prompt = ChatPromptTemplate.from_template(prompt_template)

# 3. Create a function to format the retrieved documents
def format_docs(docs):
    # Joins the 'text' of each retrieved document with a separator
    return "\n\n---\n\n".join([d["text"] for d in docs])

# 4. Build the RAG Chain
# This chains together the retrieval, document formatting, prompt, and LLM
rag_chain = (
    # This part of the chain takes the user's question
    {"context": lambda x: format_docs(retrieve_faiss_openai(x["question"], index, all_chunks)), "question": lambda x: x["question"]}
    | prompt
    | llm
    | StrOutputParser()
)

# 5. Invoke the chain with a query
query = "Are there any remote software engineer jobs that mention Python and AWS?"
response = rag_chain.invoke({"question": query})

print("--- RAG Response ---")
print(response)

# Example with another query
query_2 = "What are the responsibilities for a data scientist at Quantum Solutions?"
response_2 = rag_chain.invoke({"question": query_2})

print("\n--- RAG Response 2 ---")
print(response_2)

# API Endpoints

In [ ]:
!pip install fastapi uvicorn

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List

In [ ]:
app = FastAPI()

#Request Body

class QueryRequest(BaseModel):
  query: str
  k: int = 5

@app.post("/api/query")
def query_jobs(request: QueryRequest):
  results = retrieve_faiss_openai(request.query, index, all_chunks, k=request.k)
  return results

SyntaxError: invalid syntax (ipython-input-3865924499.py, line 1)